In [1]:
import numpy as np
import datetime as dt

from qubic.lib.Qgps import GPS, GPS_old

%matplotlib widget

RuntimeError: 'widget is not a recognised GUI loop or backend name

In [ ]:
data_path = "calsource_orientation_20240322.dat"
observation_date = np.array([dt.datetime(2024, 3, 22, 9, 47, 49), dt.datetime(2024, 3, 22, 16, 13, 49, 500000)])#np.array([dt.datetime(2024, 3, 22, 10, 8, 00)]) # 

In [ ]:
distance_between_antennas = 1
distance_calsource = 0.5
gps_old = GPS_old(data_path, distance_between_antennas, distance_calsource, observation_date)

[[-0.66504048 -0.66461026 -0.66512421 ...  1.1572724   1.15676616
   1.15338387]
 [ 0.39572939  0.39574942  0.39583976 ...  0.26029199  0.26097058
   0.26318748]
 [-0.01680668 -0.02102135 -0.02324635 ... -0.05396915 -0.05473218
  -0.05398805]]
[[ 0.1783  0.1793  0.1782 ...  0.7756  0.7746  0.774 ]
 [ 0.9331  0.9322  0.9332 ... -0.6637 -0.6628 -0.6618]
 [-0.0137 -0.015  -0.0163 ... -0.0302 -0.0303 -0.0324]]
[[-0.24337024 -0.24265513 -0.2434621  ...  0.9664362   0.96568308
   0.96369194]
 [ 0.66441469  0.66397471  0.66451988 ... -0.201704   -0.20091471
  -0.19930626]
 [-0.01525334 -0.01801067 -0.01977318 ... -0.04208457 -0.04251609
  -0.04319402]]


In [ ]:

gps = GPS(data_path, )

In [ ]:
gps.plot_system()

NameError: name 'gps' is not defined